In [5]:
import sys
import os
import torch
import matplotlib.pyplot as plt
import torchvision.transforms.functional as trans_f

sys.path.append(os.path.abspath(sys.path[0] + '/../../'))
__package__ = "deep_view_syn.notebook"
torch.cuda.set_device(2)
print("Set CUDA:%d as current device." % torch.cuda.current_device())

from ..data.spherical_view_syn import *
from ..configs.spherical_view_syn import SphericalViewSynConfig
from ..my import netio
from ..my import util
from ..my import device
from ..my import view
from ..my.gen_final import GenFinal


def load_net(path):
    config = SphericalViewSynConfig()
    config.from_id(path[:-4])
    config.SAMPLE_PARAMS['perturb_sample'] = False
    config.print()
    net = config.create_net().to(device.GetDevice())
    netio.LoadNet(path, net)
    return net


def find_file(prefix):
    for path in os.listdir():
        if path.startswith(prefix):
            return path
    return None


def load_views(data_desc_file) -> view.Trans:
    with open(data_desc_file, 'r', encoding='utf-8') as file:
        data_desc = json.loads(file.read())
        view_centers = torch.tensor(
            data_desc['view_centers'], device=device.GetDevice()).view(-1, 3)
        view_rots = torch.tensor(
            data_desc['view_rots'], device=device.GetDevice()).view(-1, 3, 3)
        return view.Trans(view_centers, view_rots)


def plot_figures(images, center):
    plt.figure(figsize=(8, 4))
    plt.subplot(121)
    util.PlotImageTensor(images['fovea_raw'])
    plt.subplot(122)
    util.PlotImageTensor(images['fovea'])

    plt.figure(figsize=(8, 4))
    plt.subplot(121)
    util.PlotImageTensor(images['mid_raw'])
    plt.subplot(122)
    util.PlotImageTensor(images['mid'])

    plt.figure(figsize=(8, 4))
    plt.subplot(121)
    util.PlotImageTensor(images['periph_raw'])
    plt.subplot(122)
    util.PlotImageTensor(images['periph'])

    # Plot Blended
    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    util.PlotImageTensor(images['blended_raw'])
    plt.subplot(122)
    util.PlotImageTensor(images['blended'])
    plt.plot([(res_full[1] - 1) / 2 + center[0] - 5, (res_full[1] - 1) / 2 + center[0] + 5],
                [(res_full[0] - 1) / 2 + center[1],
                (res_full[0] - 1) / 2 + center[1]],
                color=[0, 1, 0])
    plt.plot([(res_full[1] - 1) / 2 + center[0], (res_full[1] - 1) / 2 + center[0]],
                [(res_full[0] - 1) / 2 + center[1] - 5,
                (res_full[0] - 1) / 2 + center[1] + 5],
                color=[0, 1, 0])

Set CUDA:2 as current device.


In [6]:
#os.chdir(sys.path[0] + '/../data/__0_user_study/us_gas_all_in_one')
#os.chdir(sys.path[0] + '/../data/__0_user_study/us_mc_all_in_one')
os.chdir(sys.path[0] + '/../data/lobby_all_in_one')
print('Change working directory to ', os.getcwd())
torch.autograd.set_grad_enabled(False)

fovea_net = load_net(find_file('fovea'))
periph_net = load_net(find_file('periph'))

# Load Dataset
views = load_views('nerf_views.json')
print('Dataset loaded.')

print('views:', views.size())
#print('ref views:', ref_dataset.samples)

fov_list = [20, 45, 110]
res_list = [(128, 128), (256, 256), (256, 230)]  # (192,256)]
res_full = (1600, 1440)
gen = GenFinal(fov_list, res_list, res_full, fovea_net, periph_net,
               device=device.GetDevice())


Change working directory to  /home/dengnc/deep_view_syn/data/lobby_all_in_one
==== Config fovea ====
Net type:  nmsl
Encode dim:  10
Optimizer decay:  0
Normalize:  False
Direction as input:  False
Full-connected network parameters: {'nf': 128, 'n_layers': 4, 'skips': []}
Sample parameters {'spherical': True, 'depth_range': (1.0, 50.0), 'n_samples': 32, 'perturb_sample': False, 'lindisp': True, 'inverse_r': True}
Load net from fovea@nmsl-rgb_e10_fc128x4_d1.00-50.00_s32.pth ...
==== Config periph ====
Net type:  nnmsl
Encode dim:  10
Optimizer decay:  0
Normalize:  False
Direction as input:  False
Full-connected network parameters: {'nf': 64, 'n_layers': 4, 'skips': []}
Sample parameters {'spherical': True, 'depth_range': (1.0, 50.0), 'n_samples': 16, 'perturb_sample': False, 'lindisp': True, 'inverse_r': True}
Load net from periph@nnmsl-rgb_e10_fc64x4_d1.00-50.00_s16.pth ...
Dataset loaded.
views: [13]


In [ ]:
center_x_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for view_idx in range(10):
    center = (center_x_list[view_idx], -center_x_list[view_idx] * 1600 / 1440)
    test_view = views.get(view_idx)
    images = gen.gen(center, test_view, True)
    #plot_figures(images, center)

    util.CreateDirIfNeed('output/teasers')
    for key in images:
        util.WriteImageTensor(
            images[key], 'output/teasers/view%04d_%s.png' % (view_idx, key))
